# Classes:

In [51]:
# import pygame
from threading import Thread
import time

class Renderer():
    def __init__(self, chefe, conteudo, titulo=None, dimensoes=(800, 800), visual = True):
        self.visual = visual
        self.chefe = chefe
        self.conteudo = conteudo
        self.conteudos = [conteudo] # para trocar entre janelas
        self.iConteudoAtual = 0 # para marcar qual o atual dentre os varios
        self.titulo = titulo
        self.dimensoes = dimensoes
        self.running = True
        if not self.titulo:
            self.titulo = type(chefe).__name__   # titulo é o nome da classe
        
        self.tamanhosprite = 64
        self.escala = (self.dimensoes[0]/len(self.conteudo[0]), self.dimensoes[1]/len(self.conteudo))
        while self.escala[0] < self.tamanhosprite//8 or self.escala[1] < self.tamanhosprite//8: # redimensiona pra pp
            self.dimensoes =(int(self.dimensoes[0] *1.1), int(self.dimensoes[1]*1.1))
            self.escala = (self.dimensoes[0]/len(self.conteudo[0]), self.dimensoes[1]/len(self.conteudo))

        if visual:
            self.carregarSprites()

        # cria uma thread que roda o pygame
        mostrador = Thread(target=self.mostrar)

        # Inicia a thread
        mostrador.start()

    def addConteudo(self,conteudo):
        self.conteudos.append(conteudo)

    def desligar(self):
        self.running = False

    def carregarSprites(self):
        self.sprites = dict()
        self.sprites["path"] = pygame.transform.scale(pygame.image.load("imgs/path.png"), (int(self.escala[0]), int(self.escala[1])))
        self.sprites["wall"] = pygame.transform.scale(pygame.image.load("imgs/wall.png"), (int(self.escala[0]), int(self.escala[1])))
        self.sprites["goal"] = pygame.transform.scale(pygame.image.load("imgs/goal.png"), (int(self.escala[0]), int(self.escala[1])))
        self.sprites["agent"] = pygame.transform.scale(pygame.image.load("imgs/agent.png"), (int(self.escala[0]), int(self.escala[1])))
        self.sprites["right"] = pygame.transform.scale(pygame.image.load("imgs/right.png"), (int(self.escala[0]), int(self.escala[1])))
        self.sprites["up"] = pygame.transform.scale(pygame.image.load("imgs/up.png"), (int(self.escala[0]), int(self.escala[1])))
        self.sprites["left"] = pygame.transform.scale(pygame.image.load("imgs/left.png"), (int(self.escala[0]), int(self.escala[1])))
        self.sprites["down"] = pygame.transform.scale(pygame.image.load("imgs/down.png"), (int(self.escala[0]), int(self.escala[1])))


    def caracterParaSprite(self, caracter):
        match caracter:
            case "wall":
                return "#"
            case "right":
                return "→"
            case "up":
                return "↑"
            case "left":
                return "←"
            case "down":
                return "↓"
            case _:
                return " "

    def mostrar(self):
        # renderiza o ambiente
        if self.visual:
            pygame.init()
            self.screen = pygame.display.set_mode(self.dimensoes)
            pygame.display.set_caption(self.titulo)
            self.screen.fill((0, 0, 0))

        if self.visual:
            while self.running:
                pygame.time.delay(10)  # delay de 10ms
                # Botao de fechar
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        self.running = False
                    # se apertar "p"
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_p:
                            self.iConteudoAtual = 1
                            self.conteudo = self.conteudos[self.iConteudoAtual]

                    if event.type == pygame.KEYUP:
                        if event.key == pygame.K_p:
                            self.iConteudoAtual = 0
                            self.conteudo = self.conteudos[self.iConteudoAtual]

                # limpa a tela
                self.screen.fill((0,0,0))
                
                # desenha o conteudo
                for i in range(len(self.conteudo)):
                    for j in range(len(self.conteudo[0])):
                        celula = self.conteudo[i][j]
                        # se o conteudo de celula estiver no dicionario de sprites
                        if celula in self.sprites:
                            objeto = celula
                        else:
                            objeto = self.chefe.simbolos[celula]
                        self.screen.blit(self.sprites[objeto], (j*self.escala[0], i*self.escala[1]))

                # Atualizar a tela
                pygame.display.update()
        else:
            for linha in self.conteudo:
                for celula in linha:
                    print(self.caracterParaSprite(celula), end="")
                print()

        


In [52]:
import random

class Agent():
    acoes = ['up', 'down', 'left', 'right']
    def __init__(self, gamma = 0.9) -> None:
        self.x = None
        self.y = None
        self.gamma = gamma

    def iniciaQ(self, formato):
        """
        livroQ é uma lista de listas de dicionarios,
        ele armazena 
        
        """
        self.livro_Q = []
        for i in range(formato[0]):
            self.livro_Q.append([])
            for _ in range(formato[1]):
                self.livro_Q[i].append(dict())
                for acao in self.acoes:
                    self.livro_Q[i][-1][acao] = float("-inf")
    
    def iniciaPolicy(self, formato, politicaAleatoria):
        """
        A policy é uma matriz de caracteres que guarda a acao principal
        a ser tomada ate o momento
        
        """
        self.policy = []
        for i in range(formato[0]):
            self.policy.append([])
            for j in range(formato[1]):
                if self.environment.simbolos[self.environment.mapaOriginal[i][j]] == "wall":
                    self.policy[i].append("wall")
                    continue
                if politicaAleatoria:
                    self.policy[i].append(random.choice(self.acoes))
                else:
                    self.policy[i].append(self.acoes[0])
        # self.render = self.environment.render.addConteudo(self.policy)

    def iniciaRetorno(self, formato):
        """
        returns é uma colecao de pares estado acao guardando um
        dicionario para armazenar o valor maximo de reforcos obtidos,
        o numero de vezes que o par estado acao foi visitado e o ultimo
        episodio em que o par estado acao foi visitado
        """
        self.returns = []
        for i in range(formato[0]):
            self.returns.append([])
            for j in range(formato[1]):
                self.returns[i].append(dict())
                for acao in self.acoes:
                    self.returns[i][j][acao] = {"value": 0, "count": 0, "lastEpisode": 0}

    def setEnvironment(self, environment):
        self.environment = environment
    
    def setPos(self, x, y):
        self.x = x
        self.y = y

    def mover(self, acao):
        return self.environment.mover(self, acao)
    
    def get_action(self):
        return self.policy[self.y][self.x]


In [53]:
import time
class Environment:
    simbolosPadrao = {"agent": '@', "wall": '#', "path": '.', "goal":'$'}
    def __init__(self, path = None) -> None:
        self.agentPos = []
        if path:
            self.mapaOriginal = self.carregarMapa(path)
        self.mapa = self.copiarMapa(self.mapaOriginal)
        # self.render = Renderer(self, self.mapa, "Ambiente", visual = False)
        self.tempoEspera = 0

    def copiarMapa(self, mapa):
        mapaCopia = []
        for linha in mapa:
            mapaCopia.append([])
            for celula in linha:
                mapaCopia[-1].append(celula)
        return mapaCopia

    def setAgent(self, agent) -> Agent:
        self.agent = agent

    def in_terminal_state(self):
        return self.mapaOriginal[self.agent.y][self.agent.x] == self.simbolosPadrao["goal"]


    def carregarMapa(self, path):
        """
        Dado o caminho path, le um arquivo txt e retorna uma matriz
        O txt consiste de uma linha contendo o numero n (número de
        linhas do mapa) e m (número de caracteres diferentes no mapa),
        seguido de m linhas explicando o que sao os caracteres no 
        arquivo e por fim n linhas contendo o mapa que sao caracteres
        """
        mapa = []
        self.simbolos = dict()
        self.reforcos = {"agent": 0, "wall": 0, "path": 0, "goal":0}
        with open(path, 'r') as arquivo:
            m, n = map(int, arquivo.readline().split())
            for _ in range(m):
                linha = arquivo.readline().split()
                self.simbolos[linha[0]] = linha[1]
                self.reforcos[linha[1]] = int(linha[2])
            for i in range(n):
                linha = arquivo.readline()
                mapa.append([])
                for j in range(len(linha)):
                    char = linha[j]
                    if char == '\n':
                        continue
                    if self.simbolos[char] == 'agent':
                        self.agentPos.append((i, j))
                        char = self.simbolosPadrao["path"]
                    mapa[-1].append(self.simbolosPadrao[self.simbolos[char]])
        return mapa
    
    def mover(self, agent, acao):
        """
        Dada uma acao, move o agente no mapa
        a acao pode ser "up", "down", "left" ou "right"
        """
        time.sleep(self.tempoEspera)
        direcao = {"up": (-1, 0), "down": (1, 0), "left": (0, -1), "right": (0, 1)}
        posicaofinal = (agent.y+direcao[acao][0], agent.x+direcao[acao][1])
        if self.mapa[posicaofinal[0]][posicaofinal[1]] != self.simbolosPadrao["wall"]:
            # seta a posicao atual como caminho
            self.mapa[agent.y][agent.x] = self.mapaOriginal[agent.y][agent.x]
            
            # seta a posicao final como o agente
            self.mapa[posicaofinal[0]][posicaofinal[1]] = self.simbolosPadrao["agent"]
            
            # atualiza a posicao do agente
            agent.setPos(x = posicaofinal[1], y = posicaofinal[0])
        # retorna o reforco da posicao final 
        return self.reforcos[self.simbolos[self.mapaOriginal[agent.y][agent.x]]]

    def util(self, pos, acao):
        """
        Dada uma posicao e uma acao, retorna o que tem na posicao destino
        """
        direcao = {"up": (-1, 0), "down": (1, 0), "left": (0, -1), "right": (0, 1)}
        posicaofinal = (pos[0]+direcao[acao][0], pos[1]+direcao[acao][1])
        return self.simbolos[self.mapa[posicaofinal[0]][posicaofinal[1]]] != "wall"

    def get_size(self):
        return len(self.mapa), len(self.mapa[0])
    
    def setAgentPos(self, i, j):
        if self.agent.y != None and self.agent.x != None:
            self.mapa[self.agent.y][self.agent.x] = self.mapaOriginal[self.agent.y][self.agent.x]
        self.agent.setPos(j, i)
        self.mapa[i][j] = self.simbolosPadrao["agent"]

    def get_metrics(self):
        pontuacao = 0
        for pos in self.agentPos:
            pontuacao += max(self.agent.livro_Q[pos[0]][pos[1]].values())
        return pontuacao/len(self.agentPos)

In [54]:
class LearningStrategy():
    def train(self, episodes):
        pass

    def setup(self, environment, agent):
        pass

In [55]:
class LinearFunctionApproximation():
    def __init__(self) -> None:
        pass

In [56]:
import random
import time
class MonteCarlo(LearningStrategy):
    def __init__(self) -> None:
        pass

    def setup(self, environment, agent):
        self.environment = environment
        self.agent = agent
    
    def train(self, episodes, politicaAleatoria = True, chanceExploracao = 0):
        # Initialize
        formato = self.environment.get_size()
        self.agent.iniciaPolicy(formato, politicaAleatoria)
        Renderer(self, self.agent.policy, "Politica", visual = False)
        self.agent.iniciaQ(formato)
        self.agent.iniciaRetorno(formato)
        self.metricas = []
        for ep in range(1,episodes+1):
            if ep % (episodes//10) == 0:
                print(f"Episodio {ep} de {episodes} ", end = "\t")

            # escolhe posicao aleatoria valida para o agente
            while True:
                estado = (random.randrange(0, formato[0]), random.randrange(0, formato[1]))
                if self.environment.mapaOriginal[estado[0]][estado[1]] in {self.environment.simbolosPadrao["path"], self.environment.simbolosPadrao["goal"]}:
                    break
            # escolhe uma acao diferente da dita pela politica atual
            for _ in range(len(self.agent.acoes)*2):    # limite maximo de tentativas
                acao = random.choice(self.agent.acoes)
                if acao != self.agent.policy[estado[0]][estado[1]]:
                    # se a acao nao te leva para uma parede
                    if self.environment.util(estado, acao):
                        break
            else:
                acao = self.agent.policy[estado[0]][estado[1]]
            self.episode(estado, acao, max_steps= formato[1]*formato[0], chanceExploracao = chanceExploracao)
            g = 0
            for t in range(len(self.agent.lembrancas)-1, -1, -1): 
                memoria = self.agent.lembrancas[t]  # memoria = (estado, acao, reforco)
                g = self.agent.gamma*g + memoria[2]
                # verifica se o par estado acao ja foi inserido em returns
                if self.agent.returns[memoria[0][0]][memoria[0][1]][memoria[1]]["lastEpisode"] != ep:
                    self.agent.returns[memoria[0][0]][memoria[0][1]][memoria[1]]["lastEpisode"] = ep
                    self.agent.returns[memoria[0][0]][memoria[0][1]][memoria[1]]["value"] += g
                    self.agent.returns[memoria[0][0]][memoria[0][1]][memoria[1]]["count"] += 1
                    media = self.agent.returns[memoria[0][0]][memoria[0][1]][memoria[1]]["value"]/self.agent.returns[memoria[0][0]][memoria[0][1]][memoria[1]]["count"]
                    self.agent.livro_Q[memoria[0][0]][memoria[0][1]][memoria[1]] = media
                    self.agent.policy[memoria[0][0]][memoria[0][1]] = max(self.agent.acoes, key = lambda acao: self.agent.livro_Q[memoria[0][0]][memoria[0][1]][acao])    # recebe a acao que maximiza o valor de Q
            self.metricas.append(self.environment.get_metrics())
            if ep % (episodes//10) == 0:
                print(f"Pontuacao total nos pontos destacados: {self.metricas[-1]}")

    def episode(self, estado, acao, max_steps, chanceExploracao = 0):
        step_count = 0
        self.agent.lembrancas = []
        self.environment.setAgentPos(estado[0], estado[1])
        while step_count < max_steps:  # enquanto nao estiver em um estado terminal
            step_count +=1  # incrementa o numero de passos
            posAnterior = (self.agent.y, self.agent.x)
            reward = self.environment.mover(self.agent,acao) # realiza a acao e recebe a recompensa
            self.agent.lembrancas.append((posAnterior, acao, reward)) # guarda o passo
            if random.randint(0, 100)< (1-chanceExploracao)*100:
                acao = self.agent.get_action() # escolhe uma acao de acordo com a politica
            else:
                for _ in range(len(self.agent.acoes)*2):    # limite maximo de tentativas
                    acao = random.choice(self.agent.acoes)
                    # se a acao nao te leva para uma parede
                    if self.environment.util(estado, acao):
                        break
            if self.environment.in_terminal_state():
                break

In [57]:
class QLearning():
    def __init__(self) -> None:
        pass

In [58]:
class SARSA():
    def __init__(self) -> None:
        pass

In [59]:
class Professor():
    learningStrategys = {"Monte Carlo": MonteCarlo(), "Q-Learning": QLearning(), "SARSA": SARSA(), "Linear Function Approximation": LinearFunctionApproximation()}
    
    def __init__(self, caminhoSala = "sala", learningStrategy = None, nEpisodios = 1000, chanceExploracao = 0):
        self.learningStrategy = self.learningStrategys[learningStrategy]
        self.env = Environment(path = caminhoSala)
        self.aluno = Agent(gamma = 0.9)
        self.env.setAgent(self.aluno)
        self.aluno.setEnvironment(self.env)
        self.learningStrategy.setup(self.env, self.aluno)
        self.chanceExploracao = chanceExploracao
        self.ensinar(nEpisodios)

    def ensinar(self, nEpisodios):
        self.learningStrategy.train(nEpisodios, chanceExploracao = self.chanceExploracao)
        Renderer(self, self.aluno.policy, "Politica", visual = False)

In [62]:
if __name__ == '__main__':
    sala = input("Digite o numero da sala: ")
    professor = Professor(caminhoSala=f"sala{sala}.txt", learningStrategy="Monte Carlo", nEpisodios=100000, chanceExploracao = 0.3)


##########
#←→→↓↓↑#↓#
#↑####←#←#
#←↑↑↑#→↑→#
####→#####
#←←↓→←→→←#
#↑↓###↓#←#
#↑←#←→↑#↑#
#→↑↑→#↓←←#
##########
Episodio 10000 de 100000 	Pontuacao total nos pontos destacados: 12.644875610974536
Episodio 20000 de 100000 	Pontuacao total nos pontos destacados: 12.72440182021445
Episodio 30000 de 100000 	Pontuacao total nos pontos destacados: 12.803759914497409
Episodio 40000 de 100000 	Pontuacao total nos pontos destacados: 12.882370095207287
Episodio 50000 de 100000 	Pontuacao total nos pontos destacados: 12.91912595496232
Episodio 60000 de 100000 	Pontuacao total nos pontos destacados: 12.952370193690857
Episodio 70000 de 100000 	Pontuacao total nos pontos destacados: 12.983027433294442
Episodio 80000 de 100000 	Pontuacao total nos pontos destacados: 13.021535001683677
Episodio 90000 de 100000 	Pontuacao total nos pontos destacados: 13.05089296570234
Episodio 100000 de 100000 	Pontuacao total nos pontos destacados: 13.056423673776717
##########
#→→→→→↓#↓#
#↑####↓#↑#
#↑←←←#→→↑#
####↑###